In [2]:
import instaloader
import time
import random
from itertools import cycle
from datetime import datetime, timedelta
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import ApplicationBuilder, CommandHandler, CallbackQueryHandler, MessageHandler, filters
import pandas as pd
import re
import nest_asyncio
import asyncio

# Применяем патч для работы с текущим event loop
nest_asyncio.apply()

# Список аккаунтов для авторизации (логин и пароль)
accounts = [
    {'username': 'your_username1', 'password': 'your_password1'},
    {'username': 'your_username2', 'password': 'your_password2'}
]

# Циклический итератор для распределения запросов между аккаунтами
account_cycle = cycle(accounts)

# Глобальная переменная для Instaloader
L = None

# Функция авторизации в Instagram
def login_instagram():
    global L
    account = next(account_cycle)  # Переход к следующему аккаунту в списке
    L = instaloader.Instaloader()
    try:
        L.login(account['username'], account['password'])
        print(f"Авторизация прошла успешно на аккаунте: {account['username']}")
    except instaloader.exceptions.ConnectionException:
        print(f"Ошибка авторизации на аккаунте: {account['username']}. Переключаюсь на другой аккаунт.")
        login_instagram()  # Переключаемся на следующий аккаунт в случае ошибки

# Инициализация первого аккаунта
login_instagram()

# Функция для анализа профиля с обновлением прогресса
async def analyze_instagram_profile(username, update, start_date, end_date):
    global L
    try:
        await update.message.reply_text('Ищу профиль в Instagram...')
        profile = instaloader.Profile.from_username(L.context, username)
        posts = [post for post in profile.get_posts() if start_date <= post.date <= end_date]

        total_posts = len(posts)
        if total_posts == 0:
            await update.message.reply_text(f'Нет постов в диапазоне {start_date} - {end_date}.')
            return None, None

        # Собираем основную информацию о профиле
        await update.message.reply_text('Собираю основную информацию о профиле...')
        general_info = {
            "Profile URL": [f"https://instagram.com/{username}"],
            "Posts": [profile.mediacount],
            "Followers": [profile.followers],
            "Followings": [profile.followees],
            "Bio": [profile.biography]
        }

        # Вычисляем пост с наибольшим количеством лайков
        best_post = max(posts, key=lambda post: post.likes)
        general_info["Best Post URL"] = [f"https://instagram.com/p/{best_post.shortcode}/"]
        general_info["Best Post Likes"] = [best_post.likes]
        general_info["Best Post Comments"] = [best_post.comments]

        # Вычисляем частоту публикаций
        if len(posts) > 1:
            post_frequency = (posts[0].date - posts[-1].date).days / len(posts)
        else:
            post_frequency = "N/A"
        
        general_info["Post Frequency (days between posts)"] = [post_frequency]

        # Рассчитываем уровень вовлеченности
        general_info["Engagement Rate"] = [best_post.likes / profile.followers * 100] if profile.followers > 0 else 0

        # Сохраняем и отправляем таблицу с общей информацией
        df_general = pd.DataFrame(general_info)
        general_info_file = f"{username}_general_info.xlsx"
        df_general.to_excel(general_info_file, index=False)
        await update.message.reply_text('Отправляю таблицу с общей информацией...')
        await update.message.reply_document(document=open(general_info_file, 'rb'))

        # Информирование о начале сбора постов
        progress_message = await update.message.reply_text(f'Анализирую посты: 0 из {total_posts}')

        processed_posts = 0
        post_data = []
        for post in posts:
            try:
                comments = list(post.get_comments())
                hashtags = " ".join(post.caption_hashtags)
                top_comments = [f"@{comment.owner.username}: {comment.text}" for comment in comments[:10]]
                external_links = ', '.join([link for link in post.caption_mentions])

                post_date = post.date.strftime("%Y-%m-%d")
                post_time = post.date.strftime("%H:%M:%S")
                
                post_data.append({
                    "Post URL": f"https://instagram.com/p/{post.shortcode}/",
                    "Post Date": post_date,
                    "Post Time": post_time,
                    "Likes": post.likes,
                    "Comments": post.comments,
                    "Hashtags": hashtags,
                    "Post Text": post.caption,
                    "Top 10 Comments": "; ".join(top_comments),
                    "External Links": external_links
                })
                processed_posts += 1
                await progress_message.edit_text(f'Анализирую посты: {processed_posts} из {total_posts}')
                await asyncio.sleep(random.randint(1, 4))  # Асинхронная задержка
            except Exception as e:
                print(f"Error during post analysis: {str(e)}")
                login_instagram()

        # Окончание анализа постов
        await progress_message.edit_text('Анализ постов завершен. Создаю таблицу с анализом постов...')

        df_posts = pd.DataFrame(post_data)
        posts_info_file = f"{username}_posts_info.xlsx"
        df_posts.to_excel(posts_info_file, index=False)
        await update.message.reply_document(document=open(posts_info_file, 'rb'))

    except Exception as e:
        await update.message.reply_text(f'Произошла ошибка: {str(e)}')
        return None, str(e)

# Функция для извлечения username из URL или @user_name
def extract_username_from_input(input_text):
    # Проверка формата URL
    match_url = re.match(r"(?:http[s]?://)?(?:www\.)?instagram\.com/([A-Za-z0-9._]+)/?", input_text)
    if match_url:
        return match_url.group(1)
    # Проверка формата @username
    match_at = re.match(r"@([A-Za-z0-9._]+)", input_text)
    if match_at:
        return match_at.group(1)
    # Если не соответствует ни одному формату, возвращаем None
    return None

# Функция для проверки формата даты
def is_valid_date(date_text):
    try:
        datetime.strptime(date_text, '%Y-%m-%d')
        return True
    except ValueError:
        return False

# Хэндлер для старта
async def start(update: Update, context):
    keyboard = [
        [InlineKeyboardButton("Info", callback_data='info')],
        [InlineKeyboardButton("Analysis (table)", callback_data='analysis_table')],
        [InlineKeyboardButton("Analysis (ChatGPT)", callback_data='analysis_chatgpt')]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    await update.message.reply_text('Welcome to the Instagram Analysis Bot!', reply_markup=reply_markup)

# Хэндлер для обработки кнопок
async def button_handler(update: Update, context):
    query = update.callback_query
    await query.answer()

    if query.data == 'info':
        await query.message.reply_text('Привет! Этот бот анализирует Instagram профили.')
    elif query.data == 'analysis_table':
        # Отображаем кнопки для выбора диапазона времени
        keyboard = [
            [InlineKeyboardButton("1 месяц", callback_data='1_month')],
            [InlineKeyboardButton("3 месяца", callback_data='3_months')],
            [InlineKeyboardButton("Свой диапазон", callback_data='custom_range')]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        await query.message.reply_text('Выберите диапазон времени для анализа:', reply_markup=reply_markup)
        context.user_data['action'] = 'choose_range'

    elif query.data == '1_month' and context.user_data.get('action') == 'choose_range':
        # Устанавливаем диапазон в 1 месяц от текущей даты
        end_date = datetime.now()
        start_date = end_date - timedelta(days=30)
        context.user_data['start_date'] = start_date
        context.user_data['end_date'] = end_date
        context.user_data['action'] = 'analysis_table'
        await query.message.reply_text('Пожалуйста, вставьте ссылку на профиль в Instagram (например: https://instagram.com/user_name) или @user_name.')

    elif query.data == '3_months' and context.user_data.get('action') == 'choose_range':
        # Устанавливаем диапазон в 3 месяца от текущей даты
        end_date = datetime.now()
        start_date = end_date - timedelta(days=90)
        context.user_data['start_date'] = start_date
        context.user_data['end_date'] = end_date
        context.user_data['action'] = 'analysis_table'
        await query.message.reply_text('Пожалуйста, вставьте ссылку на профиль в Instagram (например: https://instagram.com/user_name) или @user_name.')

    elif query.data == 'custom_range' and context.user_data.get('action') == 'choose_range':
        # Переход к ручному вводу диапазона дат
        context.user_data['action'] = 'start_date'
        await query.message.reply_text('Пожалуйста, введите начальный диапазон даты в формате ГГГГ-ММ-ДД (например, 2024-08-19).')

    elif query.data == 'analysis_chatgpt':
        await query.message.reply_text('Эта функция пока не реализована.')

# Хэндлер для обработки сообщений
async def message_handler(update: Update, context):
    if 'action' in context.user_data:
        action = context.user_data['action']

        if action == 'start_date':
            start_date_text = update.message.text.strip()
            if is_valid_date(start_date_text):
                context.user_data['start_date'] = datetime.strptime(start_date_text, '%Y-%m-%d')
                await update.message.reply_text('Теперь введите конечный диапазон даты в формате ГГГГ-ММ-ДД (например, 2024-09-19).')
                context.user_data['action'] = 'end_date'
            else:
                await update.message.reply_text('Неверный формат даты. Пожалуйста, введите дату в формате ГГГГ-ММ-ДД.')

        elif action == 'end_date':
            end_date_text = update.message.text.strip()
            if is_valid_date(end_date_text):
                context.user_data['end_date'] = datetime.strptime(end_date_text, '%Y-%m-%d')
                await update.message.reply_text('Пожалуйста, вставьте ссылку на профиль в Instagram (например: https://instagram.com/user_name) или @user_name.')
                context.user_data['action'] = 'analysis_table'
            else:
                await update.message.reply_text('Неверный формат даты. Пожалуйста, введите дату в формате ГГГГ-ММ-ДД.')

        elif action == 'analysis_table':
            profile_input = update.message.text.strip()

            # Извлекаем username из ввода
            username = extract_username_from_input(profile_input)

            if username:
                # Получаем введенные ранее даты
                start_date = context.user_data.get('start_date')
                end_date = context.user_data.get('end_date')

                # Проверяем, что даты заданы правильно
                if start_date and end_date and start_date <= end_date:
                    await update.message.reply_text(f'Начинаю анализ профиля {username} с {start_date.strftime("%Y-%m-%d")} по {end_date.strftime("%Y-%m-%d")}...')
                    
                    # Добавление сообщения о процессе авторизации
                    await update.message.reply_text('Авторизация в Instagram...')
                    
                    # Выполняем анализ профиля
                    general_info_file, posts_info_file = await analyze_instagram_profile(username, update, start_date, end_date)

                    if general_info_file:
                        # Отправляем файл пользователю
                        await update.message.reply_document(document=open(posts_info_file, 'rb'))
                    else:
                        await update.message.reply_text(f'Произошла ошибка: {posts_info_file}')
                else:
                    await update.message.reply_text('Диапазон дат указан некорректно. Пожалуйста, начните заново.')
            else:
                await update.message.reply_text('Я сейчас не понял, что от меня требуется, пожалуйста, нажмите снова на кнопку Analysis (table) и введите профиль заново.')

            # Сбрасываем действие после завершения анализа
            context.user_data['action'] = None

# Основная функция
if __name__ == '__main__':
    application = ApplicationBuilder().token('BOT_TOKEN_TELEGA').build()

    # Регистрируем обработчики для команд и сообщений
    application.add_handler(CommandHandler('start', start))
    application.add_handler(CallbackQueryHandler(button_handler))
    application.add_handler(MessageHandler(filters.TEXT, message_handler))

    # Запуск бота
    application.run_polling()

Авторизация прошла успешно на аккаунте: ilyxa_stulov


No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/Users/iliastulov/miniconda3/lib/python3.12/site-packages/telegram/ext/_application.py", line 1335, in process_update
    await coroutine
  File "/Users/iliastulov/miniconda3/lib/python3.12/site-packages/telegram/ext/_handlers/basehandler.py", line 157, in handle_update
    return await self.callback(update, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/b6/kcqg7wpx5_q3ngxl5ff5t3mr0000gn/T/ipykernel_59750/2622899059.py", line 223, in message_handler
    general_info_file, posts_info_file = await analyze_instagram_profile(username, update, start_date, end_date)
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: cannot unpack non-iterable NoneType object


RuntimeError: Cannot close a running event loop